In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense,Embedding,GRU,LSTM


In [0]:
data=pd.read_csv('movie_data.csv',encoding='utf-8')

In [4]:
data.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [0]:
x_train, x_test, y_train, y_test = train_test_split(data['review'],data['sentiment'], test_size=0.33, random_state=42)

In [0]:
x_train=x_train.tolist()
x_test=x_test.tolist()
y_test=y_test.tolist()
y_train=y_train.tolist()

In [0]:
#Tokens
tokens=Tokenizer()
totalvocab=x_train+x_test
tokens.fit_on_texts(totalvocab)

#Pad sequences
max_len=max([len(seq.split())  for seq in totalvocab])
vocab=len(tokens.word_index)+1


#To sequence
x_train_trans=tokens.texts_to_sequences(x_train)
x_test_trans=tokens.texts_to_sequences(x_test)

#Pad
x_train_pad=pad_sequences(x_train_trans,maxlen=max_len,padding='post')
x_test_pad=pad_sequences(x_test_trans,maxlen=max_len,padding='post')

In [8]:
EmbeddingDim=100

model=Sequential()
model.add(Embedding(vocab,EmbeddingDim,input_length=max_len))
model.add(LSTM(units=32,dropout=0.2,return_sequences=True))
model.add(LSTM(units=64,dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2470, 100)         12425300  
_________________________________________________________________
lstm_1 (LSTM)                (None, 2470, 32)          17024     
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 12,467,221
Trainable params: 12,467,221
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_train_pad,y_train,batch_size=128,epochs=100,validation_data=(x_test_pad,y_test),verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 33500 samples, validate on 16500 samples
Epoch 1/100
